In [328]:
import re
# from nltk.corpus import stopwords
from scipy import sparse
import numpy as np
import holidays
from utils import ExtractCalendar
from datetime import date
import re
import pandas as pd
from datetime import date
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


## Load the data from csv and format correctly

In [2]:
invoice_data = pd.read_csv('./input/invoice_data_final.csv')

baskets = invoice_data.loc[invoice_data.index.repeat(invoice_data['SalesQTY'])].groupby('Invoiceno')['ProductCode'].apply(list)
products_in_basket = invoice_data.loc[invoice_data.index.repeat(invoice_data['SalesQTY'])].groupby('Invoiceno')['ProductName'].apply(list)
other_elements = invoice_data[['Invoiceno','PrepareDate', 'WeekEndFlag', 'DayTimeFlag']].drop_duplicates().set_index(['Invoiceno'])
other_elements['PrepareDate'] = pd.to_datetime(other_elements.PrepareDate)
basket_corpus = pd.concat([baskets, products_in_basket, other_elements], axis=1)
daytime_dict = {key: i for i, key in enumerate(basket_corpus.DayTimeFlag.unique())}
basket_corpus.DayTimeFlag = basket_corpus.DayTimeFlag.map(daytime_dict)
basket_corpus

,ProductCode,ProductName,PrepareDate,WeekEndFlag,DayTimeFlag
Invoiceno,,,,,
D0071501010177,"[2500464, 2805315, 2805913, 2805913, 3100861]","[AARONG UHT Full Cream Liquid Milk 500ml, Prot...",2015-01-01 09:44:08.857,0,0
D0071501010182,"[2700164, 2700327, 2702241, 2500234]","[Radhuni Panch Foran 50 gm, Pran Thai Chilli S...",2015-01-01 09:56:30.700,0,0
D0071501010192,"[3901391, 4203132, 4203156, 2802216, 2802216, ...","[S201011 Party Spray Cragy Ribbon-PS1/2, S2010...",2015-01-01 10:08:11.857,0,0
D0071501010248,"[2805453, 2700122, 2400450, 2805452, 2700154, ...","[IFAD Eggy Instant Noodles Chicken 65Gm, Radhu...",2015-01-01 11:18:25.920,0,0
D0071501010311,"[2801349, 2801347, 2802229, 2300267]","[Golden Harvest Dal Puri 454gm (10 Pcs), GH De...",2015-01-01 12:07:57.460,0,0
...,...,...,...,...,...
D0072209191015,"[2816455, 2816455, 2815330, 2815330, 2815277, ...","[Fresh Healthy Choice Salted Biscuit 100g, Fre...",2022-09-19 21:14:46.067,0,3
D0072209191029,"[2603361, 2603361, 2603361, 2900006, 2900006, ...","[Wheel Laundry Soap 125g, Wheel Laundry Soap 1...",2022-09-19 21:39:07.430,0,3
D0072209191033,"[3017647, 3000680, 3811722, 3000382]","[Vaseline Mosquito Defence Original 50ml, Clos...",2022-09-19 21:42:29.560,0,3


## Add festival/Holiday flag

In [3]:
calendar = ExtractCalendar([2015, 2016, 2017, 2018, 2019, 2021, 2022], 'bangladesh')
def is_festival(_date):
    return _date in calendar.festivals

basket_corpus = pd.concat([basket_corpus, basket_corpus.PrepareDate.rename('FestivalFlag').apply(is_festival).astype(int)], axis=1)

/tmp/ipykernel_6244/3142944248.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return _date in calendar.festivals


In [4]:
basket_corpus[['ProductCode', 'ProductName', 'WeekEndFlag', 'DayTimeFlag', 'FestivalFlag']]

,ProductCode,ProductName,WeekEndFlag,DayTimeFlag,FestivalFlag
Invoiceno,,,,,
D0071501010177,"[2500464, 2805315, 2805913, 2805913, 3100861]","[AARONG UHT Full Cream Liquid Milk 500ml, Prot...",0,0,0
D0071501010182,"[2700164, 2700327, 2702241, 2500234]","[Radhuni Panch Foran 50 gm, Pran Thai Chilli S...",0,0,0
D0071501010192,"[3901391, 4203132, 4203156, 2802216, 2802216, ...","[S201011 Party Spray Cragy Ribbon-PS1/2, S2010...",0,0,0
D0071501010248,"[2805453, 2700122, 2400450, 2805452, 2700154, ...","[IFAD Eggy Instant Noodles Chicken 65Gm, Radhu...",0,0,0
D0071501010311,"[2801349, 2801347, 2802229, 2300267]","[Golden Harvest Dal Puri 454gm (10 Pcs), GH De...",0,0,0
...,...,...,...,...,...
D0072209191015,"[2816455, 2816455, 2815330, 2815330, 2815277, ...","[Fresh Healthy Choice Salted Biscuit 100g, Fre...",0,3,0
D0072209191029,"[2603361, 2603361, 2603361, 2900006, 2900006, ...","[Wheel Laundry Soap 125g, Wheel Laundry Soap 1...",0,3,0
D0072209191033,"[3017647, 3000680, 3811722, 3000382]","[Vaseline Mosquito Defence Original 50ml, Clos...",0,3,0


In [270]:
invoice_data[['SalesQTY']] = invoice_data[['SalesQTY']].astype(int)
product_code2name =invoice_data[['ProductCode','ProductName']].drop_duplicates().set_index('ProductCode').to_dict()['ProductName']
product_name2code =invoice_data[['ProductCode','ProductName']].drop_duplicates().set_index('ProductName').to_dict()['ProductCode']
unique_product_code = [value for key, value in invoice_data[['ProductCode']].drop_duplicates(ignore_index=True).to_dict()['ProductCode'].items()]
unique_product_name = [value for key, value in invoice_data[['ProductName']].drop_duplicates(ignore_index=True).to_dict()['ProductName'].items()]

In [272]:
product_code2name
product_name2code
unique_product_name

['AARONG UHT Full Cream Liquid Milk 500ml',
 'Protiva Chui Pitha 200g',
 'Satej Coconut Naru 200g',
 'X-Shonalika Chicken Dressed Large (Pcs)',
 'Radhuni Panch Foran 50 gm',
 'Pran Thai Chilli Sauce 340gm',
 'Pran Soy Sauce 650ml',
 'No.1Sweetened condensed milk 400 gm',
 'S201011 Party Spray Cragy Ribbon-PS1/2',
 'S201011 Love Balloon Thai 20 Pcs BL 1/7',
 'S201011 Balloon Thai BL 1/2',
 'MARS Fun Size 18gm',
 'IFAD Eggy Instant Noodles Chicken 65Gm',
 'Radhuni Turmeric Regular 50 gm',
 'Shwapno Sugar (P) 1Kg',
 'IFAD Eggy Instant Noodles Masala 65Gm',
 'Radhuni Chilli 100g',
 'ACI Pure Salt 1kg',
 'Harpic Power Plus 500 ML',
 'Ispahani Mirzapore Bestleaf 200g(P.Bag)',
 'Mama Instant Noodles Chicken Flav. 248g',
 'Rupchanda Soyabean Oil 2ltr',
 'Teer Suji 500gm',
 'Golden Harvest Dal Puri 454gm (10 Pcs)',
 'GH Deshi Paratha 20pcs 1300g',
 'Ruchi Barbecue Chanachur 170 gm',
 'Mac Coffee Mix 3B1 Original 20g 25pcs',
 'Shan Karachi Beef Biryani Mix 60g',
 'Cerelac3 Fruits, Wheat & Milk 4

In [76]:
product_codes_by_SalesQTY = invoice_data.groupby('ProductCode')['SalesQTY'].sum('SalesQTY').sort_values(ascending=False).reset_index()['ProductCode'].to_list()

In [7]:
unique_product_name_dict

{'AARONG UHT Full Cream Liquid Milk 500ml': 0,
 'Protiva Chui Pitha 200g': 1,
 'Satej Coconut Naru 200g': 2,
 'X-Shonalika Chicken Dressed Large (Pcs)': 3,
 'Radhuni Panch Foran 50 gm': 4,
 'Pran Thai Chilli Sauce 340gm': 5,
 'Pran Soy Sauce 650ml': 6,
 'No.1Sweetened condensed milk 400 gm': 7,
 'S201011 Party Spray Cragy Ribbon-PS1/2': 8,
 'S201011 Love Balloon Thai 20 Pcs BL 1/7': 9,
 'S201011 Balloon Thai BL 1/2': 10,
 'MARS Fun Size 18gm': 11,
 'IFAD Eggy Instant Noodles Chicken 65Gm': 12,
 'Radhuni Turmeric Regular 50 gm': 13,
 'Shwapno Sugar (P) 1Kg': 14,
 'IFAD Eggy Instant Noodles Masala 65Gm': 15,
 'Radhuni Chilli 100g': 16,
 'ACI Pure Salt 1kg': 17,
 'Harpic Power Plus 500 ML': 18,
 'Ispahani Mirzapore Bestleaf 200g(P.Bag)': 19,
 'Mama Instant Noodles Chicken Flav. 248g': 20,
 'Rupchanda Soyabean Oil 2ltr': 21,
 'Teer Suji 500gm': 22,
 'Golden Harvest Dal Puri 454gm (10 Pcs)': 23,
 'GH Deshi Paratha 20pcs 1300g': 24,
 'Ruchi Barbecue Chanachur 170 gm': 25,
 'Mac Coffee Mix 3B

In [8]:
unique_product_code_dict

{2500464: 0,
 2805315: 1,
 2805913: 2,
 3100861: 3,
 2700164: 4,
 2700327: 5,
 2702241: 6,
 2500234: 7,
 3901391: 8,
 4203132: 9,
 4203156: 10,
 2802216: 11,
 2805453: 12,
 2700122: 13,
 2400450: 14,
 2805452: 15,
 2700154: 16,
 2400029: 17,
 2600082: 18,
 2300193: 19,
 2805707: 20,
 2400017: 21,
 2400066: 22,
 2801349: 23,
 2801347: 24,
 2802229: 25,
 2300267: 26,
 2701861: 27,
 2200033: 28,
 2200147: 29,
 2400023: 30,
 2500068: 31,
 2700060: 32,
 3101315: 33,
 2800071: 34,
 2800400: 35,
 2700155: 36,
 3000833: 37,
 2800249: 38,
 2400084: 39,
 2400513: 40,
 2805166: 41,
 2801405: 42,
 2600069: 43,
 3333053: 44,
 3003439: 45,
 3333888: 46,
 2807734: 47,
 3333944: 48,
 2602652: 49,
 3002319: 50,
 2700105: 51,
 2400868: 52,
 2400025: 53,
 2600008: 54,
 2400441: 55,
 2600079: 56,
 2600015: 57,
 2600111: 58,
 3000798: 59,
 3000831: 60,
 2400007: 61,
 2400019: 62,
 2701535: 63,
 2702154: 64,
 2805179: 65,
 2801287: 66,
 2500100: 67,
 2500704: 68,
 2500002: 69,
 2901015: 70,
 2900936: 71,
 2

## Create basket iterator

In [9]:
# baskets = [TaggedDocument(product, i) for key, value in pro]
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents[0]
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

TaggedDocument(words=['human', 'interface', 'computer'], tags=[0])

In [210]:

class tagged_basket_iterator:
    def __init__(self, tb):
        self._df = tb._df
        self._index = 0

    def __next__(self):
        total_len = len(self._df)
        if self._index >= total_len:
            raise StopIteration
        value = self._df.iloc[self._index]
        tag = self._df.index[self._index]
        self._index += 1
        return TaggedDocument(words=value, tags=[tag])

class tagged_baskets:
    def __init__(self, df):
        self._df = df

    def __iter__(self):
        return tagged_basket_iterator(self)

## Make embeddings from Product Name

In [313]:
basket_df = basket_corpus['ProductName']
basket_class = tagged_baskets(basket_df)
model_name = Doc2Vec(basket_class,
                dm=0,
                dbow_words=1,
                vector_size=128, #Tuned parameter
                window=500,
                epochs=10, #Tuned parameter
                hs=1,
                workers=4,
                min_count=50 # We have 2M baskets. And if the item does not even exist in 50 baskets, the embedding quality will be poor. Better to drop than create a poor embedding
           )

## Save and Load model

In [314]:
# Save model
model_name.save('train-by-name')
# Load saved model
model_name = Doc2Vec.load('train-by-name')

In [326]:
unique_product_name[2]

'Satej Coconut Naru 200g'

In [327]:
index = model_name.wv.key_to_index[product_code2name[unique_product_code[2]]]
[(name, probability) for (name, probability) in model_name.wv.most_similar(model_name.wv.vectors[index])]

[('Satej Coconut Naru 200g', 1.0),
 ('J.M Special Chira Bhaja 200gm', 0.6699640154838562),
 ('Satej Shahi Khurma 150g', 0.6391971707344055),
 ('Oitijja Murali 300g', 0.6301323771476746),
 ('JMF.Special Nimki 100g', 0.6299176812171936),
 ('Satej Coconut Chips 100g', 0.6187284588813782),
 ('J.M Special Dal(Mug) Bhaja 200gm', 0.6187019944190979),
 ('Satej Badam Tana 1 Pcs', 0.6081887483596802),
 ('Satej Fry Badam Vaza 100g', 0.6067774891853333),
 ('J.M Special Badam Bhaja 200gm', 0.5982539653778076)]

## Make embeddings from Product Code

In [315]:
basket_df = basket_corpus['ProductCode']
model_code = Doc2Vec(tagged_baskets(basket_df),
                    dm=0,
                    dbow_words=1,
                    vector_size=128, #Tuned parameter
                    window=500,
                    epochs=10, #Tuned parameter
                    hs=1,
                    workers=4,
                    min_count=50 # We have 2M baskets. And if the item does not even exist in 50 baskets, the embedding quality will be poor. Better to drop than create a poor embedding
               )

## Save and Load model

In [319]:
# Save model
model_code.save('train-by-code')
# Load saved model
model_code = Doc2Vec.load('train-by-code')

## Generate context words

In [322]:
unique_product_name[2]

'Satej Coconut Naru 200g'

In [324]:
index = model_code.wv.key_to_index[product_name2code[unique_product_name[2]]]
[(product_code2name[code], probability) for (code, probability) in model_code.wv.most_similar(model_code.wv.vectors[index])]

[('Satej Coconut Naru 200g', 1.0),
 ('Oitijja Murali 300g', 0.6677554249763489),
 ('JMF.Special Nimki 100g', 0.6544570326805115),
 ('J.M Special Chira Bhaja 200gm', 0.6524658203125),
 ('J.M Special Badam Bhaja 200gm', 0.6515520215034485),
 ('J.M Special Dal(Mug) Bhaja 200gm', 0.6346307992935181),
 ('Satej Shahi Khurma 150g', 0.6094410419464111),
 ('Satej Chira Vaza 200g', 0.6005433201789856),
 ('Satej Muri Moya 12pcs Pack', 0.5938534140586853),
 ('Satej Oithoya 100g', 0.5884043574333191)]

[(2400346, 0.001988597),
 (2813350, 0.0011625664),
 (2702979, 0.0011463923),
 (3372903, 0.0011119131),
 (3015597, 0.0011029616),
 (2800589, 0.0010963166),
 (2809948, 0.0010556945),
 (2811951, 0.0010240444),
 (2500946, 0.0009893847),
 (2808518, 0.0009785686)]

In [240]:
context_products = []
for p_name, probability in model_code.predict_output_word(basket_corpus['ProductName'].tolist()[120][0:1]):
    # print(product_name_map[pcode], probability)
    print(p_name, probability)

# context_products

Olitalia Pure Olive Oil 100 ml 0.001910133
meril glycerin 60 gm 0.0015893462
Oral-B Sensitive Tooth Brush Extra Soft 0.0013102316
Vaseline P.Petroleum Original Jelly 100m 0.0011111205
Danish Pineapple S.Cream Biscuits 63g 0.0010631962
Betaseptic Hand Rub 250ml 0.0010589052
Bisk Club Shero Cream Biscuit 85g 0.0010538296
Johnsons Baby Cream Jar 50g(imp) 0.0010461066
YC Lip Care F.S. Strawberry Lip Gel 3.8g 0.0010138301
RS Extra Virgen Olive Oil Tin 200ml 0.0009849513


In [239]:
for pcode, probability in model2.predict_output_word(basket_corpus['ProductCode'].tolist()[120][0:1]):
    print(product_name_map[pcode], probability)

Olitalia Pure Olive Oil 100 ml 0.001988597
Shwapno Boot Vaza 100g(B) 0.0011625664
Ahmed Garlic Paste 100g 0.0011463923
S201165 Neema Thai 0.0011119131
Tresemme Keratin Smooth Shampoo 580ml 0.0011029616
Richi Beef Kabab 12 pcs 0.0010963166
Danish Pineapple S.Cream Biscuits 63g 0.0010556945
Globe Tiffin Strawb. Biscuits 25g 0.0010240444
Pran Premium Butter 200g (Box) 0.0009893847
Knorr Classic Chicken Corn Soup 24g 0.0009785686


In [119]:
for pcode in basket_corpus['ProductCode'].tolist()[1000][3:4]:
    print(product_name_map[pcode])

Pepsi Pet Bottle 2000ml


In [167]:
(2400346, 0.001988597),
 (2813350, 0.0011625664),
 (2702979, 0.0011463923),
 (3372903, 0.0011119131),
 (3015597, 0.0011029616),
 (2800589, 0.0010963166),
 (2809948, 0.0010556945),
 (2811951, 0.0010240444),
 (2500946, 0.0009893847),
 (2808518, 0.0009785686)

(233864, 6)

## Save or load the model

In [178]:
# Save model
model.save('saved_model.model')

In [179]:
# Load saved model
model2 = Doc2Vec.load('saved_model')

In [320]:
unique_product_name[2]

'Satej Coconut Naru 200g'

In [321]:
index = model_code.wv.key_to_index[product_name2code[unique_product_name[2]]]
[(product_code2name[code], probability) for (code, probability) in model_code.wv.most_similar(model_code.wv.vectors[index])]

[('Satej Coconut Naru 200g', 1.0),
 ('Oitijja Murali 300g', 0.6677554249763489),
 ('JMF.Special Nimki 100g', 0.6544570326805115),
 ('J.M Special Chira Bhaja 200gm', 0.6524658203125),
 ('J.M Special Badam Bhaja 200gm', 0.6515520215034485),
 ('J.M Special Dal(Mug) Bhaja 200gm', 0.6346307992935181),
 ('Satej Shahi Khurma 150g', 0.6094410419464111),
 ('Satej Chira Vaza 200g', 0.6005433201789856),
 ('Satej Muri Moya 12pcs Pack', 0.5938534140586853),
 ('Satej Oithoya 100g', 0.5884043574333191)]

In [310]:
index = model_code.wv.key_to_index[product_code2name[unique_product_code[2]]]
[(name, probability) for (name, probability) in model_code.wv.most_similar(model_code.wv.vectors[index])]

[('Satej Coconut Naru 200g', 1.0),
 ('J.M Special Dal(Mug) Bhaja 200gm', 0.6655147075653076),
 ('Oitijja Murali 300g', 0.6228452920913696),
 ('JMF.Special Nimki 100g', 0.6131789684295654),
 ('J.M Special Badam Bhaja 200gm', 0.6129363775253296),
 ('Satej Shahi Khurma 150g', 0.6109143495559692),
 ('Satej Coconut Chips 100g', 0.5929478406906128),
 ('Satej Nimki 200g', 0.5929191708564758),
 ('J.M Special Dabli Bhaja 200gm', 0.5891609191894531),
 ('J.M Special Chira Bhaja 200gm', 0.5804669857025146)]

In [209]:
# model.make_cum_table()
model.prepare_vocab()
model.similarity_unseen_docs(str(basket_corpus['ProductCode'].tolist()[1]).strip('[]').split(', '), str(basket_corpus['ProductCode'].tolist()[-10]).strip('[]').split(', '))



0.07981811

In [172]:
print(product_name_map[basket_corpus['ProductCode'].tolist()[1000][3]], '\n')
for pcode, similarity in model.wv.most_similar(basket_corpus['ProductCode'].tolist()[1000][3:4]):
    print(product_name_map[pcode], similarity)


# for inv_code, similarity in model.dv.most_similar(['D0071501040269']):
#     print(basket_corpus.loc[inv_code, :][1], similarity)

# for pcode, similarity in model.dv.most_similar(['D0071501040269']):
#     print(product_name_map[pcode], similarity)
# model.infer_vector([2300006, 2300007, 2300008, 2300009])

Pepsi Pet Bottle 2000ml 

Banana Sun Facing (Surjamukhi Kola) 1.420524001121521
X-Roast(Promo)Pcs 1.3727611303329468
S201011 Birthday Cap Long BCL 1/2M 1.169028878211975
Banana Red Sobri (Agni Sobri) 0.8870706558227539
AG Food Luchi 10Pcs 350g 0.8562366962432861
Cauliflower (Fulcopy) PCS 0.8260385394096375
Biotique Coconut Hair Oil 200ml 0.7967800498008728
Naturally Fair Soap 100g 0.7532094120979309
Shwapno Toilet Tissue Gold 0.7513838410377502
Koyeler Mangsho Dressed 55g(Pcs) 0.7372216582298279


In [ ]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [2]:


invoice_data = pd.read_csv('./input/invoice_data_2.csv')
invoice_data[['SalesQTY']] = invoice_data[['SalesQTY']].astype(int)
product_name_map =invoice_data[['ProductCode','ProductName']].drop_duplicates().set_index('ProductName').to_dict()['ProductCode']
unique_product_code_dict = {value: key for key, value in invoice_data[['ProductCode']].drop_duplicates(ignore_index=True).to_dict()['ProductCode'].items()}
unique_product_name_dict = {value: key for key, value in invoice_data[['ProductName']].drop_duplicates(ignore_index=True).to_dict()['ProductName'].items()}

baskets = invoice_data.loc[invoice_data.index.repeat(invoice_data['SalesQTY'])].groupby('Invoiceno')['ProductCode'].apply(list)
products_in_basket = invoice_data.loc[invoice_data.index.repeat(invoice_data['SalesQTY'])].groupby('Invoiceno')['ProductName'].apply(list)
basket_corpus = pd.concat([baskets, products_in_basket], axis=1)

# window = 2

# table = invoice_data.groupby(['Invoiceno', 'ProductCode'])['SalesQTY'].apply(list)
# invoice_data.groupby(['Invoiceno', 'ProductCode'])['SalesQTY'].apply(list)
# invoice_data.groupby('Invoiceno')['ProductCode'].apply(list)

# invoice_data.groupby('Invoiceno')['ProductName'].apply(list)
# invoice = invoice_data[]
# invoice_data.loc[invoice_data["Invoiceno"]==invoice]
# x = []
# for invoice in invoice_data['Invoiceno'].unique():
#     x.append(invoice_data.loc[invoice_data["Invoiceno"]==invoice]['ProductCode'])


In [31]:
# invoice_data
# date(2022, 5,2)
bd_holidays = holidays.BD(years=2021)
# date(2021, 5, 3) in bd_holidays
# bd_holidays.items()
for date, name in sorted(bd_holidays.items()):
    print(date, name)

2021-02-21 International Mother's language Day
2021-03-17 Sheikh Mujibur Rahman's Birthday and Children's Day
2021-03-26 Independence Day
2021-04-14 Bengali New Year's Day
2021-05-01 May Day
2021-08-15 National Mourning Day
2021-12-16 Victory Day


In [18]:
type(baskets)

pandas.core.series.Series

In [8]:
window = max(baskets.apply(len))
# window = 10
product_lists = []
name_lists = []
all_products = []
all_product_names = []

for basket, product_names in zip(baskets, products_in_basket):
    # text = clean_text(text)
    all_products += basket
    all_product_names += product_names
    for i, (product, name) in enumerate(zip(basket, product_names)):
        for w in range(window):
            if i+1+w < len(basket):
                product_lists.append([product]+[basket[(i+1+w)]])
                name_lists.append([name]+[product_names[(i+1+w)]])


In [11]:
product_lists

[[2800294, 2700815],
 [2800294, 2500002],
 [2800294, 2500002],
 [2800294, 2400365],
 [2800294, 2400365],
 [2800294, 2700125],
 [2800294, 2700155],
 [2800294, 2700155],
 [2800294, 2700155],
 [2800294, 2700737],
 [2800294, 2700739],
 [2800294, 2700745],
 [2800294, 3002785],
 [2800294, 2801328],
 [2800294, 2801328],
 [2800294, 2801328],
 [2800294, 2400019],
 [2800294, 2500067],
 [2800294, 2500067],
 [2800294, 2500067],
 [2800294, 2500067],
 [2800294, 2600272],
 [2700815, 2500002],
 [2700815, 2500002],
 [2700815, 2400365],
 [2700815, 2400365],
 [2700815, 2700125],
 [2700815, 2700155],
 [2700815, 2700155],
 [2700815, 2700155],
 [2700815, 2700737],
 [2700815, 2700739],
 [2700815, 2700745],
 [2700815, 3002785],
 [2700815, 2801328],
 [2700815, 2801328],
 [2700815, 2801328],
 [2700815, 2400019],
 [2700815, 2500067],
 [2700815, 2500067],
 [2700815, 2500067],
 [2700815, 2500067],
 [2700815, 2600272],
 [2500002, 2500002],
 [2500002, 2400365],
 [2500002, 2400365],
 [2500002, 2700125],
 [2500002, 27

[[2800294, 2700815],
 [2800294, 2500002],
 [2800294, 2500002],
 [2800294, 2400365],
 [2800294, 2400365],
 [2800294, 2700125],
 [2800294, 2700155],
 [2800294, 2700155],
 [2800294, 2700155],
 [2800294, 2700737],
 [2800294, 2700739],
 [2800294, 2700745],
 [2800294, 3002785],
 [2800294, 2801328],
 [2800294, 2801328],
 [2800294, 2801328],
 [2800294, 2400019],
 [2800294, 2500067],
 [2800294, 2500067],
 [2800294, 2500067],
 [2800294, 2500067],
 [2800294, 2600272],
 [2700815, 2500002],
 [2700815, 2500002],
 [2700815, 2400365],
 [2700815, 2400365],
 [2700815, 2700125],
 [2700815, 2700155],
 [2700815, 2700155],
 [2700815, 2700155],
 [2700815, 2700737],
 [2700815, 2700739],
 [2700815, 2700745],
 [2700815, 3002785],
 [2700815, 2801328],
 [2700815, 2801328],
 [2700815, 2801328],
 [2700815, 2400019],
 [2700815, 2500067],
 [2700815, 2500067],
 [2700815, 2500067],
 [2700815, 2500067],
 [2700815, 2600272],
 [2500002, 2500002],
 [2500002, 2400365],
 [2500002, 2400365],
 [2500002, 2700125],
 [2500002, 27

In [16]:
unique_product_code_dict

{2800294: 0,
 2700815: 1,
 2500002: 2,
 2400365: 3,
 2700125: 4,
 2700155: 5,
 2700737: 6,
 2700739: 7,
 2700745: 8,
 3002785: 9,
 2801328: 10,
 2400019: 11,
 2500067: 12,
 2600272: 13,
 2801390: 14,
 2600241: 15,
 2802211: 16,
 2801004: 17,
 2600021: 18,
 2800393: 19,
 2800360: 20,
 2803106: 21,
 2801402: 22,
 2800525: 23,
 2803115: 24,
 2801347: 25,
 2802623: 26,
 2802937: 27,
 2803123: 28,
 2803093: 29,
 2803121: 30,
 2803124: 31,
 2500066: 32,
 2300011: 33,
 2800303: 34,
 2400475: 35,
 2800286: 36,
 2500035: 37,
 3002216: 38,
 2600016: 39,
 2300304: 40,
 2800112: 41,
 3000383: 42,
 2500031: 43,
 2801652: 44,
 2900525: 45,
 2200026: 46,
 2700063: 47,
 2700053: 48,
 2500088: 49,
 2300365: 50,
 2600101: 51,
 2700326: 52,
 2801886: 53,
 2700479: 54,
 2800243: 55,
 2100227: 56,
 2400033: 57,
 2801632: 58,
 2400029: 59,
 2200011: 60,
 2400059: 61,
 2800934: 62,
 3001377: 63,
 2400075: 64,
 2803088: 65,
 2300179: 66,
 2300271: 67,
 2300249: 68,
 2300364: 69,
 2800249: 70,
 2500011: 71,
 3

In [ ]:
# One-hot-encoding
n_products = len(unique_product_code_dict )
products = list(unique_product_code_dict.keys())

X = []
Y = []

for code_pair in product_lists:
    X.append((code_pair[0] == np.array(products))*1)
    Y.append((code_pair[1] == np.array(products))*1)

X = np.asarray(X)
Y = np.asarray(X)

In [78]:
products_in_basket[0]


['Meridian Real Thai Chicken Chips 18(±3)g',
 'Provati Cumin (Jeera) Whole 200g',
 'Aarong white Curd 500 ml',
 'Aarong white Curd 500 ml',
 'Confidence Salt 1 kg',
 'Confidence Salt 1 kg',
 'Radhuni Turmeric Regular 500 gm',
 'Radhuni Chilli 200g',
 'Radhuni Chilli 200g',
 'Radhuni Chilli 200g',
 'Provati Cinnamon (Daruchini) 100g',
 'Provati Cloves (Lobongo) 50g',
 'Provati Raisins (Kishmish) 100g',
 'Savlon Aloe Vera Handwash 200ml (Refill)',
 'X-Bounty -2-2 Chocolate 57gm (B)',
 'X-Bounty -2-2 Chocolate 57gm (B)',
 'X-Bounty -2-2 Chocolate 57gm (B)',
 'Rupchanda Soyabean Oil 5ltr',
 'Milk Vita Liquid Milk 1000 Ml',
 'Milk Vita Liquid Milk 1000 Ml',
 'Milk Vita Liquid Milk 1000 Ml',
 'Milk Vita Liquid Milk 1000 Ml',
 'Super Napthalene\xa0(30Pcs+Pack)']

In [28]:
window = 2
word_lists = []
all_products = []
all_product_names = []
for basket, product_in_basket in zip(baskets, products_in_basket):
    all_products += basket


    # for i,
# dict(all_products)
def create_unique_product_dict(products:list) -> dict:
    products = list(products)
    unique_product_dict = {product: i for i, product in enumerate(list(set(products)))}
    return unique_product_dict
create_unique_product_dict(all_products)



TypeError: unhashable type: 'list'

In [ ]:
window = 2
word_lists = []
all_text = []

for text in texts:
    text = clean_text(text)
    all_text += text
    for i, word in enumerate(text):
        for w in range(window):
            if i + 1 + w < len(text):
                word_lists.append([word] + [text[(i + 1 + w)]])
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])
            # print(i,word)
word_lists


In [227]:
product_corpus

,ProductCode,ProductName
Invoiceno,,
D0071103230271,"[2800294, 2700815, 2500002, 2500002, 2400365, ...","[Meridian Real Thai Chicken Chips 18(±3)g, Pro..."
D0071103230301,"[2801390, 2600241, 2802211, 2802211, 2801004, ...","[Azad Pan Pasanda Special 100 gm, Fire Box (12..."
D0071103240075,"[2801402, 2800525, 2800525, 2803115, 2803115, ...","[Aftab Crispy Chicken Wings 500 gm, Harvest Ri..."
D0071103240146,"[2802937, 2803123, 2803123, 2803093, 2803121, ...","[Cadbury Dairy Milk 49±4g(uk), Knorr Chicken ..."
D0071103250232,"[2500066, 2500066, 2500066, 2500066, 2500002, ...","[Milk Vita Liquid Milk 500 Ml, Milk Vita Liqui..."
...,...,...
D0072209191015,"[2816455, 2816455, 2815277, 2815277, 2815277, ...","[Fresh Healthy Choice Salted Biscuit 100g, Fre..."
D0072209191029,"[2500902, 2816053, 2603361, 2603361, 2603361, ...","[Aarong Laban 500ml, Mama Ins.Cup Noodles Spic..."
D0072209191033,"[3000680, 3000382, 3017647, 3811722]","[CloseUp Ever Fresh Menthol Fresh 100g, B.dhar..."


In [219]:
products_name

Invoiceno
D0071103230204                           [Johnson Baby Powder 200g]
D0071103250120          [X-Farlin Wet Wipes (10 PCs)  DT - 004A(B)]
D0071103260132    [Johnson Milk+Rice Baby Bath 200ml MY, Meril B...
D0071103260462             [Angel Feeding Bottle 240 ml RXA-8A2 PP]
D0071103270022          [X-Farlin Wet Wipes (35 PCs)  DT - 005B(B)]
                                        ...                        
D0072209190778                   [Freshmaker Baby Wet Wipes 120Pcs]
D0072209190929    [Kodomo Baby Shampoo Gentle Soft 200ml, Dr. C....
D0072209190950                    [Pepsodent Orange Toothpaste 45g]
D0072209190971                [Parachute Just For Baby Powder 200g]
D0072209191020                            [Savlon Baby Wipes 60Pcs]
Name: ProductName, Length: 53681, dtype: object

Invoiceno
D0071103230204                               [2100108]
D0071103250120                               [2100410]
D0071103260132                      [2100081, 2100110]
D0071103260462                               [2100227]
D0071103270022                               [2100414]
                                  ...                 
D0072209190778                               [2101922]
D0072209190929    [2101457, 2102155, 2100705, 2101284]
D0072209190950                               [2102234]
D0072209190971                               [2102084]
D0072209191020                               [2101954]
Name: ProductCode, Length: 53681, dtype: object